In [1]:
#!/usr/bin/env python3
"""
Этап 1: Выбор бактерий с разным GC-составом
"""

# Анализ таблицы с бактериями
# Из таблицы видно, что хорошие кандидаты:

# 1. Yersinia pestis - средний GC-состав (47,58%)
# Accession: CP016273.1 (хромосома)


# 2. Streptomyces albus- низкий GC-состав (72,65%)
# Accession: CP033071.1 (хромосома)
#!/usr/bin/env python3
"""
Этап 2: Загрузка последовательностей хромосом
"""

import urllib.request
import gzip
from io import StringIO

def download_chromosome(accession):
    """
    Загружает последовательность хромосомы из ENA
    """
    url = f"https://www.ebi.ac.uk/ena/browser/api/fasta/{accession}"
    
    print(f"Загрузка {accession}...")
    
    try:
        # Отправляем запрос
        request = urllib.request.Request(url)
        request.add_header('Accept-Encoding', 'gzip')
        
        with urllib.request.urlopen(request) as response:
            # Читаем данные
            data = response.read()
            
            # Распаковываем gzip если нужно
            if response.info().get('Content-Encoding') == 'gzip':
                data = gzip.decompress(data)
            
            # Декодируем в текст
            fasta_text = data.decode('utf-8')
            
            # Парсим FASTA
            lines = fasta_text.strip().split('\n')
            header = lines[0]
            sequence = ''.join(lines[1:]).upper()
            
            print(f"Загружено: {header}")
            print(f"Длина последовательности: {len(sequence)} bp")
            
            return sequence, header
            
    except Exception as e:
        print(f"Ошибка при загрузке {accession}: {e}")
        return None, None

# Загружаем обе хромосомы
print("=" * 60)
print("ЗАГРУЗКА ХРОМОСОМ")
print("=" * 60)

# Бактерия 1: Yersinia pestis
chromosome1_seq, chromosome1_header = download_chromosome("CP016273.1")

# Бактерия 2: Streptomyces albus 
chromosome2_seq, chromosome2_header = download_chromosome("CP033071.1")

# Проверяем GC-состав загруженных последовательностей
def calculate_gc_content(sequence):
    """Вычисляет GC-состав последовательности"""
    if not sequence:
        return 0
    
    g_count = sequence.count('G')
    c_count = sequence.count('C')
    total = len(sequence)
    
    gc_content = (g_count + c_count) / total * 100
    return gc_content

print("\n" + "=" * 60)
print("АНАЛИЗ GC-СОСТАВА")
print("=" * 60)

gc1 = calculate_gc_content(chromosome1_seq)
gc2 = calculate_gc_content(chromosome2_seq)

print(f"1. {chromosome1_header}")
print(f"   GC-состав: {gc1:.2f}%")
print(f"   Длина: {len(chromosome1_seq)} bp")
print()
print(f"2. {chromosome2_header}")
print(f"   GC-состав: {gc2:.2f}%")
print(f"   Длина: {len(chromosome2_seq)} bp")
print()
print(f"Разница в GC-составе: {abs(gc1 - gc2):.2f}%")

# Сохраняем последовательности в файлы
with open("chromosome1.fasta", "w") as f:
    f.write(f">{chromosome1_header}\n")
    # Записываем по 70 символов в строке (стандартный FASTA формат)
    for i in range(0, len(chromosome1_seq), 70):
        f.write(chromosome1_seq[i:i+70] + "\n")

with open("chromosome2.fasta", "w") as f:
    f.write(f">{chromosome2_header}\n")
    for i in range(0, len(chromosome2_seq), 70):
        f.write(chromosome2_seq[i:i+70] + "\n")

print("\nПоследовательности сохранены в файлы:")
print("  - chromosome1.fasta")
print("  - chromosome2.fasta")

ЗАГРУЗКА ХРОМОСОМ
Загрузка CP016273.1...
Загружено: >ENA|CP016273|CP016273.1 Yersinia pestis strain Cadman chromosome, complete genome.
Длина последовательности: 4602429 bp
Загрузка CP033071.1...
Загружено: >ENA|CP033071|CP033071.1 Streptomyces albus strain ZD11 chromosome, complete genome.
Длина последовательности: 8317371 bp

АНАЛИЗ GC-СОСТАВА
1. >ENA|CP016273|CP016273.1 Yersinia pestis strain Cadman chromosome, complete genome.
   GC-состав: 47.58%
   Длина: 4602429 bp

2. >ENA|CP033071|CP033071.1 Streptomyces albus strain ZD11 chromosome, complete genome.
   GC-состав: 72.65%
   Длина: 8317371 bp

Разница в GC-составе: 25.07%

Последовательности сохранены в файлы:
  - chromosome1.fasta
  - chromosome2.fasta
